I wasn't able to answer this question completely and I don't have prior knowlege in LLM and NLP , the stuffs that I know in ML are linear and logistic regression, decision trees, neural networks,unsupervised learning (clustering and anomaly detection), Reinforcement Learning , recommender systems. I took this code from the github repo https://github.com/SriLaxmi1993/Document-Genie-using-RAG-Framwork and integrated Attention mechanism with the help of Chat GPT for syntax , after reading the readme in github repo I came to know that This is a pseudo app that has to be run in streamlit but i don't know how to do that . What I felt was instead of leaving the answer for this question empty, let's try something even if that doesn't work that is something I made. so I just wanted to include the reason why I failed to complete this question and wanted to show that I have tried someting.

In [12]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import tensorflow as tf

st.set_page_config(page_title="My_first_chatbot", layout="wide")

# Define a simple dot-product attention layer
class DotProductAttention(tf.keras.layers.Layer):
    def call(self, query, key, value):
        # Compute attention scores
        scores = tf.matmul(query, key, transpose_b=True)
        # Apply softmax to get attention weights
        attention_weights = tf.nn.softmax(scores, axis=-1)
        # Apply attention weights to the value vectors
        attention_output = tf.matmul(attention_weights, value)
        return attention_output

# This is the first API key input; no need to repeat it in the main function.
api_key = st.text_input("Enter your Google API Key:", type="password", key="api_key_input")

def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks, api_key):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain_with_attention():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, google_api_key=api_key)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

def user_input_with_attention(user_question, api_key):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)
    
    # Apply attention mechanism
    attention_layer = DotProductAttention()
    attention_output = attention_layer(query=docs, key=docs, value=docs)
    
    chain = get_conversational_chain_with_attention()
    response = chain({"input_documents": attention_output, "question": user_question}, return_only_outputs=True)
    st.write("Reply: ", response["output_text"])

def main():
    st.header("AI clone chatbot💁")

    user_question = st.text_input("Ask a Question from the PDF Files", key="user_question")

    if user_question and api_key:  # Ensure API key and user question are provided
        user_input_with_attention(user_question, api_key)

    with st.sidebar:
        st.title("Menu:")
        pdf_docs = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True, key="pdf_uploader")
        if st.button("Submit & Process", key="process_button") and api_key:  # Check if API key is provided before processing
            with st.spinner("Processing..."):
                raw_text = get_pdf_text(pdf_docs)
                text_chunks = get_text_chunks(raw_text)
                get_vector_store(text_chunks, api_key)
                st.success("Done")

if __name__ == "__main__":
    main()
